<a href="https://colab.research.google.com/github/changhorang/SSAC_study/blob/main/DL3%20(PyTorch)/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC/Chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2-3_어휘_집합_구축하기

## BPE 기반 토크나이저 만들기
### 공통 실습환경
- install ratsnlp

In [ ]:
!pip install ratsnlp

     |████████████████████████████████| 53 kB 1.2 MB/s 
     |████████████████████████████████| 2.8 MB 9.0 MB/s 
     |████████████████████████████████| 57 kB 3.5 MB/s 
     |████████████████████████████████| 806 kB 25.9 MB/s 
     |████████████████████████████████| 332 kB 67.6 MB/s 
     |████████████████████████████████| 636 kB 51.6 MB/s 
     |████████████████████████████████| 132 kB 46.4 MB/s 
     |████████████████████████████████| 829 kB 29.1 MB/s 
     |████████████████████████████████| 895 kB 22.4 MB/s 
     |████████████████████████████████| 3.3 MB 33.7 MB/s 
     |████████████████████████████████| 61 kB 247 kB/s 
     |████████████████████████████████| 1.1 MB 44.6 MB/s 
     |████████████████████████████████| 96 kB 4.2 MB/s 
     |████████████████████████████████| 271 kB 23.7 MB/s 
     |████████████████████████████████| 192 kB 47.3 MB/s 
     |████████████████████████████████| 160 kB 41.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 s

### 말뭉치 내려받기 및 전처리
- 네이버 영화리뷰 NSMC

In [ ]:
from Korpora import Korpora
nsmc = Korpora.load('nsmc', force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 76.8MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 38.4MB/s]                            


In [ ]:
import os

def write_lines(path, lines):
  with open(path, 'w', encoding='utf-8') as f:
    for line in lines:
      f.write(f'{line}\n')

write_lines('/root/train.txt', nsmc.train.get_all_texts())
write_lines('/root/test.txt', nsmc.test.get_all_texts())

### GPT 토크나이저 구축
- GPT 계열 모델 토크나이저 기법 : BPE

In [ ]:
import os

os.makedirs('/content/drive/MyDrive/nlpbook/bbpe', exist_ok=True)

In [ ]:
from tokenizers import ByteLevelBPETokenizer

bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=['/root/train.txt', '/root/test.txt'], # 학습할 말뭉치를 리스트 형태로 넣기
    vocab_size=10000, # 어휘 집합 크기 조절
    special_tokens=['[PAD]'] # 특수 토큰 추가
)
bytebpe_tokenizer.save_model('/content/drive/MyDrive/nlpbook/bbpe')

['/content/drive/MyDrive/nlpbook/bbpe/vocab.json',
 '/content/drive/MyDrive/nlpbook/bbpe/merges.txt']

### BERT 토크나이저 구축

In [ ]:
import os

os.makedirs('/content/drive/MyDrive/nlpbook/wordpiece', exist_ok=True)

In [ ]:
from tokenizers import BertWordPieceTokenizer

wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=['/root/train.txt', '/root/test.txt'], # 학습할 말뭉치를 리스트 형태로 넣기
    vocab_size=10000 # 어휘 집합 크기 조절
)
wordpiece_tokenizer.save_model('/content/drive/MyDrive/nlpbook/wordpiece')

['/content/drive/MyDrive/nlpbook/wordpiece/vocab.txt']

## 2-4 토큰화하기
### GPT 입력값 만들기

In [ ]:
import tokenizers
from transformers import GPT2Tokenizer

tokenizer_gpt = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/nlpbook/bbpe')
tokenizer_gpt.pad_token = '[PAD]'

file /content/drive/MyDrive/nlpbook/bbpe/config.json not found


In [ ]:
sentences = [
  "아 더빙.. 진짜 짜증나네요 목소리",
  "흠...포스터보고 초딩영환줄...오버연기조차 가볍지 않구나",
  "별루 였다.."
]

tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĺ',
  'ì¤Ħ',
  '...',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

In [ ]:
# 실제 모델 입력 값
batch_inputs = tokenizer_gpt(
    sentences,
    padding='max_length', # 문자의 최대 길이에 맞춰 패딩
    max_length=12, # 문장의 토큰 기준 최대 길이
    truncation=True # 문장 잘림 허용옵션
)
batch_inputs

{'input_ids': [[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0], [3693, 336, 2876, 758, 2883, 5048, 806, 336, 9875, 875, 2960, 7292], [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]}

In [ ]:
# input_ids: 토큰화 결과를 토대로 index화 (indexing)
batch_inputs['input_ids']

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 5048, 806, 336, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
# attention_mask: 단어 인 곳 = 1 / padding부분 = 0
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

### BERT 입력값 만들기

In [ ]:
from transformers import BertTokenizer

tokenizer_bert = BertTokenizer.from_pretrained('/content/drive/MyDrive/nlpbook/wordpiece', do_lower_case=False)

file /content/drive/MyDrive/nlpbook/wordpiece/config.json not found


In [ ]:
sentences = [
  "아 더빙.. 진짜 짜증나네요 목소리",
  "흠...포스터보고 초딩영환줄...오버연기조차 가볍지 않구나",
  "별루 였다.."
]

tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]
tokenized_sentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영환',
  '##줄',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['별루', '였다', '.', '.']]

In [ ]:
# 실제 입력
batch_inputs = tokenizer_bert(
    sentences,
    padding='max_length',
    max_length=12,
    truncation=True
)

batch_inputs

{'input_ids': [[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0], [2, 997, 16, 16, 16, 2609, 2045, 2796, 4917, 1263, 16, 3], [2, 3274, 9507, 16, 16, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

In [ ]:
batch_inputs['input_ids']

[[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 4917, 1263, 16, 3],
 [2, 3274, 9507, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [ ]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [ ]:
# token_type_ids: BERT모델은 문서 혹은 문장을 입력 받음 / 입력받는 segment의 정보를 의미
batch_inputs['token_type_ids']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]